# 1) Import library

In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel  # NOT NEEDED FOR THE APP
import ast


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


# 2) Import files 

In [27]:
# profiles 
df_profiles = pd.read_csv("https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/profiles_clean.csv")

In [28]:
# synopsis embedding MiniLM
df_MiniLM = pd.read_json('https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/synopsis_embedding.json')

In [ ]:
# NOT NEEDED FOR THE APP :  synopsis embedding Jina_1024   
df_jina_1024= pd.read_json('https://anime-recommendation-engine.s3.eu-west-3.amazonaws.com/data/synopsis_embedding_jina_1024.json')

# 3) preprocess

In [29]:
def preprocess_user(df):
    df["favorites_anime"] = df["favorites_anime"].apply(ast.literal_eval)
    df_favorites = df[["profile", "favorites_anime"]].copy().explode("favorites_anime")
    df_favorites = df_favorites.dropna(subset=["favorites_anime"])
    df_favorites["favorites_anime"] = df_favorites["favorites_anime"].astype("int64")

    return df_favorites

In [30]:
df_favorites = preprocess_user(df_profiles)

# MiniLM

In [31]:
Model_MiniLM = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# similarity >= 0.5  
def search_closest_by_content_simi_50(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = model.encode(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity
        closest = similarity_df[similarity_df['similarity'] >= 0.5].sort_values(by='similarity', ascending=False)

        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df


# Top5_similarity  # NOT NEEDED FOR THE APP : 
def search_closest_by_content_top_5(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = model.encode(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity.
        closest = similarity_df.sort_values(by='similarity', ascending=False).head(5)


        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df


In [ ]:
new_content = """Despite being bullied, scorned, and oppressed all of his life, a 34-year-old shut-in still found the resolve to attempt something heroic—only for it to end in a tragic accident. But in a twist of fate, he awakens in another world as Rudeus Greyrat, starting life again as a baby born to two loving parents.

Preserving his memories and knowledge from his previous life, Rudeus quickly adapts to his new environment. With the mind of a grown adult, he starts to display magical talent that exceeds all expectations, honing his skill with the help of a mage named Roxy Migurdia. Rudeus learns swordplay from his father, Paul, and meets Sylphiette, a girl his age who quickly becomes his closest friend.

As Rudeus' second chance at life begins, he tries to make the most of his new opportunity while conquering his traumatic past. And perhaps, one day, he may find the one thing he could not find in his old world—love."""


filter = 'synopsis_embedding'
df_emb = df_MiniLM
df_favorites = df_favorites
model = Model_MiniLM

# call the function and store in DF   # NOT NEEDED FOR THE APP : 
Top_5_df = pd.DataFrame(search_closest_by_content_top_5(new_content, model, df_emb, df_favorites, filter))
display(Top_5_df.shape[0])


# call the function and store in DF
simi_50_df = pd.DataFrame(search_closest_by_content_simi_50(new_content, model, df_emb, df_favorites, filter))
display(simi_50_df.shape[0])


246

2154

 # jina emb = 1024  (abandonned)

In [10]:
# Initialize the model
Model_jina_1024 = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True).to('cuda')

/home/lyxor/miniconda3/envs/DeepLearn/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/home/lyxor/miniconda3/envs/DeepLearn/lib/python3.10/site-packages/flash_attn/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout, *args):


In [11]:
# Function to get embeddings
def get_embeddings(text):
    embeddings = model.encode([text], task="text-matching")
    return embeddings[0]

# Top5_similarity
def search_closest_by_content_jina(content, model, df_emb, df_favorites, filter):

        content = re.sub("[^A-Za-z]+", " ", str(content)).lower()
        content = get_embeddings(content)

        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([content], list(df_emb[filter]))[0]

        # Store similarity
        similarity_df = pd.DataFrame({'uid': df_emb['uid'], 'similarity': similarities})

        # filter by similarity. given_uid exclude
        closest = similarity_df.sort_values(by='similarity', ascending=False).head(5)


        df_merged = df_favorites.merge(closest, left_on='favorites_anime', right_on='uid', how='inner')
        grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

        return grouped_df

In [16]:
filter = 'synopsis_embedding'
df_emb = df_jina_1024
df_favorites = df_favorites
model = Model_jina_1024


# call the function and store in DF
jina_df = pd.DataFrame(search_closest_by_content_jina(new_content, model, df_emb, df_favorites, filter))
display(jina_df.shape[0])

393

### we will stay on the miniLM embedding, for calculation and standardise with other functions.  Similarity >= 0.5

---

In [ ]:
# ATM,  NOT NEEDED FOR THE APP
def closest_similarity(target, df, filter):
        # cosine similarity : given embedding VS all embeddings
        similarities = cosine_similarity([target], list(df[filter]))[0]
        similarity_df = pd.DataFrame({'uid': df['uid'], 'similarity': similarities})
        similarity_df =  similarity_df.sort_values(by='similarity', ascending=False)

        return similarity_df

In [ ]:
# ATM,  NOT NEEDED FOR THE APP

new_content = """Despite being bullied, scorned, and oppressed all of his life, a 34-year-old shut-in still found the resolve to attempt something heroic—only for it to end in a tragic accident. But in a twist of fate, he awakens in another world as Rudeus Greyrat, starting life again as a baby born to two loving parents.

Preserving his memories and knowledge from his previous life, Rudeus quickly adapts to his new environment. With the mind of a grown adult, he starts to display magical talent that exceeds all expectations, honing his skill with the help of a mage named Roxy Migurdia. Rudeus learns swordplay from his father, Paul, and meets Sylphiette, a girl his age who quickly becomes his closest friend.

As Rudeus' second chance at life begins, he tries to make the most of his new opportunity while conquering his traumatic past. And perhaps, one day, he may find the one thing he could not find in his old world—love."""

MiniLM = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

filter = 'synopsis_embedding'

def generate_diffusion_list(target, model, df_emb, df_favorites, filter):

    target = re.sub("[^A-Za-z]+", " ", str(target)).lower()
    target = model.encode(target)

    closest = closest_similarity(target, df_emb, filter)

    df_merged = df_favorites.merge(closest[closest['similarity'] >= 0.5], left_on='favorites_anime', right_on='uid', how='inner')
    grouped_df = df_merged.groupby('profile').agg({'uid': list}).reset_index()

    return grouped_df

diffusion_df = generate_diffusion_list(new_content, MiniLM, df_MiniLM, df_favorites, filter)
diffusion_df.shape[0]

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
